In [1]:
#Import the libraries
import s3fs
import numpy as np
#from   dask.distributed import Client
#import dask.array as da

In [2]:
#Function to collect and save in a list the files that are in the informed period of interest
def get_files(s_year,e_year,s_day,e_day,s_hour,e_our):
    print('Getting files')
    aux = []
    for y in range(s_year,e_year+1):
        for d in range(s_day,e_day):
            #The range of the variable 'd' will determine the days of the product
            for j in range(s_hour,e_our):

                try: 
                     FD = fs.ls('noaa-goes16/ABI-L2-FDCF/'+str(y)+'/'+str(d).zfill(3)+'/'+str(j).zfill(2)+'/') # list of 6 files for 2020 day 228, UTC time 15:00, 15:10, 15:20, ..., 15:50
                     aux = np.append(aux,FD)
                except FileNotFoundError as e:
                    print(f"FileNotFoundError file {'noaa-goes16/ABI-L2-FDCF/'+str(y)+'/'+str(d).zfill(3)+'/'+str(j).zfill(2)+'/'}: {e}. Skipping this file.")
                    continue  # Skip to the next file in the list
                
               
    return aux

In [3]:
# Open an ABI netCDF4 data file

# Initialize S3 file system
fs = s3fs.S3FileSystem(anon=True, use_listings_cache=True)

#Collect a example GOES-East data
# list of 6 files for 2024 day 2056, UTC time 15:00, 15:10, 15:20, ..., 15:50
#files = fs.ls('noaa-goes16/ABI-L2-AODF/2024/'+str(256).zfill(3)+'/'+str(15).zfill(2)+'/') 
#files

In [21]:
#Using the function to get the files from days and hours specified
start_year,end_year,start_day,end_day,start_hour,end_our = 2020,2020,210,211,19,20
data_list= get_files(start_year,end_year,start_day,end_day,start_hour,end_our)
data_list

Getting files


array(['noaa-goes16/ABI-L2-FDCF/2020/210/19/OR_ABI-L2-FDCF-M6_G16_s20202101900199_e20202101909507_c20202101910073.nc',
       'noaa-goes16/ABI-L2-FDCF/2020/210/19/OR_ABI-L2-FDCF-M6_G16_s20202101910199_e20202101919507_c20202101920052.nc',
       'noaa-goes16/ABI-L2-FDCF/2020/210/19/OR_ABI-L2-FDCF-M6_G16_s20202101920199_e20202101929507_c20202101930056.nc',
       'noaa-goes16/ABI-L2-FDCF/2020/210/19/OR_ABI-L2-FDCF-M6_G16_s20202101930199_e20202101939507_c20202101940032.nc',
       'noaa-goes16/ABI-L2-FDCF/2020/210/19/OR_ABI-L2-FDCF-M6_G16_s20202101940199_e20202101949506_c20202101950047.nc',
       'noaa-goes16/ABI-L2-FDCF/2020/210/19/OR_ABI-L2-FDCF-M6_G16_s20202101950198_e20202101959506_c20202102000014.nc'],
      dtype='<U108')

In [22]:
files = data_list[0]
files

'noaa-goes16/ABI-L2-FDCF/2020/210/19/OR_ABI-L2-FDCF-M6_G16_s20202101900199_e20202101909507_c20202101910073.nc'

In [23]:
#Define output directory for the data
datadir = '/home/jovyan/Article_review/Data/'

In [24]:
prodbase = files.split('/')[5][:23]
starttime = files.split('/')[5].split('_')[3]
year,julian,hhmm=starttime[1:5],starttime[5:8],starttime[8:12]
date_info=year+'-'+julian+'-'+hhmm
fileshtitle = prodbase+'-'+date_info
fileshtitle

'OR_ABI-L2-FDCF-M6_G16_s-2020-210-1900'

In [18]:
#Loop through the files selected previously
for i in range(0,len(files)):
    #Gather the date-time information from the file
    prodbase = files[i].split('/')[5][:23]
    starttime = files[i].split('/')[5].split('_')[3]
    year,julian,hhmm=starttime[1:5],starttime[5:8],starttime[8:12]
    date_info=year+'-'+julian+'-'+hhmm
    fileshtitle = prodbase+'-'+date_info
    with fs.open(files[i], 'rb') as f:
        with open(datadir+fileshtitle+'.nc', 'wb') as local_f:
            local_f.write(f.read())

IndexError: list index out of range

In [25]:
#Gather the date-time information from the file
prodbase = files.split('/')[5][:23]
starttime = files.split('/')[5].split('_')[3]
year,julian,hhmm=starttime[1:5],starttime[5:8],starttime[8:12]
date_info=year+'-'+julian+'-'+hhmm
fileshtitle = prodbase+'-'+date_info
with fs.open(files, 'rb') as f:
    with open(datadir+fileshtitle+'.nc', 'wb') as local_f:
        local_f.write(f.read())
